<a href="https://colab.research.google.com/github/Nomadyb/AttritionPredict/blob/main/timeSeriesAnomalyLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Anomaly Detection </h1>
<p>

  Anomali tespiti nadir olaylarınbulunmasını ve tanımlanmasını ifade eder.
  Bazı uygulamalar arasında banka dolandıcılığı tespiti,tıbbi görüntülemede tümör tespiti ve yazılı metindeki hatalar yer alır.
  Denetimli ve denetimsiz bir çok yaklaşım kullanılır.
  <br>
   **Tek sınıf DVM'ler**
  <br>
   **Bayes Ağları**
  <br>
   **küme Analizi**
  <br>
   **Nöral Ağlar**
  <br>

  S&P 500 endeksindeki anomalileri tespit etmek için bir LSTM autencoder Siner Ağı kullanılmıştır.
</p>

<h3>Yeni veri noktası için hata (yeniden yapılandırma hatası) belirli bir eşiğin üzerindeyse, örneği anomali olarak etiketleriz<h3>



In [ ]:
!gdown --id 10vdMg_RazoIatwrT7azKFX4P02OebU76 --output spx.csv

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

df = pd.read_csv("spx.csv",parse_dates=["date"],index_col="date")
df


Data Preprocessing


df.iloc[0:train_size] -> ilk girişten itibaren train_size indesine kadar olan bölümü alır. ve train adında yeni bir dataframe oluşturur.

df.iloc[train_size:len(df)] -> train_Size indeksinden dataframein sonuna kadar olan bölümü alır ve test frame'ine dönüştürür.

train_Size değerimizi df'in %95 olarak tanımladık.
Yani train kısmı %95 iken, test kısmı %5 olur.
overfitting ihtimalini azaltmak için böyle bir tercihte bulunuruz

In [ ]:
train_size = int(len(df)*0.95)
test_size = len(df)- train_size
train,test = df.iloc[0:train_size],df.iloc[train_size:len(df)]
print(train.shape,test.shape)


şimdi eğitim verilerini kullanarak verileri yeniden ölçeklendireceğiz ve aynı dönüşümü test verilerine uygulayacağız.


> Ölçeklendirme modellerin daha iyi sonuç vermesini sağlar.Amacımız verilerdeki büyük farklılıklar olduğunda modelimizde oluşabilecek problemleri ortadan kaldırmaktır. "StandarScaler" her bir özelliğin ortalama değerini 0 ve standart sapmasını 1 olacak şekilde ayarlar.Fit fonksiyonu "train" veri setindeki "close" özelliği üzerinde hesaplama yapar ve bu hesaplama sonucunda öğrenilen parametreleri (ortalama ve standart sapma ) içinde saklar.
Sonrasında "transform" fonksiyonu bu öğrenilen parametrelerle "train" ve test" veri setindeki "close özelliğini normalize eder. bu şekilde uyum hedeflenir.



In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train[['close']])
train['close'] = scaler.transform(train[['close']])
test['close'] = scaler.transform(test[['close']])

Şimdi verilerin Alt dizelere ayrılması sağlanır.

---

Bu fonksiyonun ana maacı bir zaman serisi veri setinden özellikler ve etiketler oluşturmayı sağlamaktır.Giriş olarak X(bağımsız değişkenler) ve y(bağımlı değişkenler) alır ve "time_steps" kadar önceki gözlemlerle yeni bir veri seti oluşturur.Bu tip veri düzenlemesi genellikle zaman serisi tahmin problemlerinde , özellikle de derin öğrenme tabanlı modellemelerde LONG Short Term Memory kullanılır.

In [ ]:
!pip install numpy

In [ ]:
import numpy  as np
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

#bu fonksiyon list compheresion yöntemi ile yazılabilir.

In [ ]:
def list_compher(X,y,time_steps=1):
  Xs = [X.iloc[i:i+ time_steps ].values for i in range (len(X)-time_steps)]
  ys = [y.iloc[i+time_steps] for i in range(len(y) - time_steps)]
  return np.array(Xs),np.array(ys)


Şimdi memory yapısı için geçmişe dönük 30 günlük veri içeren sekanslar oluşturacağız.

create_dataset fonksiyonu kullanılmaktadır
Zaman serisi için test ve eğitim test verileri oluşturulmaktadırç
Fonksiyon 4 değişken için çıktı üretir.
Boyutu TIME_STEPS kadar olur.

In [ ]:
TIME_STEPS = 30

X_train,y_train = create_dataset(
    train[["close"]],
    train.close,
    TIME_STEPS
)


X_test,y_test = create_dataset(
    test[["close"]],
    test.close,
    TIME_STEPS
)

print(X_train.shape)








# LSTM autoencoder in Keras


---
**Burada amacımız LSTM nöral ağ yapısını oluşturmaktır.**
<br>
**1.katman LSTM katmandır units parametresi LSTM katmanındaki nöral sayısını belirtir.**
<br>
**2.katman dropout yapar bu ağın ezberleme yani overfitting olasılığını azaltmak için kullanılır. rate=0.2 parametresi her eğtim için rastgele olarak close olarak belirlenen nöron oranını belirtir.**
<br>
**3.katman bir repeatvector katmanıdır.Burada önceki katmanın çıktısını belirli sayıda tekrarlama yapılır eğer sonraki katman bir girdi alıyorsa yaralıdır.**
<br>
**4.katman başka bir LSTM katmanıdır. burada return_sequences=True parametresi LSTM katmanının her time serisinde çıktı üretmesini sağlar overfitting önlemeyi amaçlar.**
<br>
**6.katman ise TimeDistributed yapar bir dense (tam bağlantılı) katmanı tüm time series adımlarında uygular**
<br>
**Son olarak "model.compile(loss="mae", optimiez="adam") modeli derler.Modelin eğitilmesinden önce gerçekleşir. "loss"="mae" parametresi gerçekleşen hataları ölçmek için kullanılcak fonksiyonu belirtir. "optimizer"="adam" parametresi modelin ağırlıklarını güncellemek için kullanacağı optimizasyon algoritmasını belirtir.  **

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:



model = keras.Sequential()
model.add(keras.layers.LSTM(
    units=64,
    input_shape=(X_train.shape[1], X_train.shape[2])
))



model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.RepeatVector(n=X_train.shape[1]))
model.add(keras.layers.LSTM(units=64, return_sequences=True))
model.add(keras.layers.Dropout(rate=0.2))
model.add(
  keras.layers.TimeDistributed(
    keras.layers.Dense(units=X_train.shape[2])



  )
)



model.compile(loss='mae', optimizer='adam')

X_train , y_train :eğtim verisi ve karşılık gelen etiketlerimizdir.Modelimizde amacımız X_train verisine göre y_train labelllarını tahmin etmektir.
<br>
<br>
epochs=10:Modelin eiğtim verileri üzerinde kaç kez geçiş yapılacağını belirler.
<br>
<br>
batch_size=32 Model her güncelleme adımında 32 örneği aynı anda işlemektedir.size büyüdükçe eğtiim hızlanır ama  ama bellek ihtiyacı artar.
<br>
<br>
validation_split=0.1 eğitim verilerini %10'unu doğrulama yani validation seti olarak ayırır.Bu modelin eğtim sürecindeki performansını izlemek için kullanılır.Model bu doğrulama verilerini eğitimde kullanmaz.Bu veri seti üzerindeki performansı sadece epoch sonunda değerlendirir.
<br>
<br>
shuffle=False eğitim verilerinin her epoch başında karıştırılıp karıştırılmayacağını belirler.Time Series  tahmini gibi sıralı verilerle çalışırken shuffle=False olarak kullanılırç
amacımız history çıktısını yani eğitim sürecini değerlendirmektir.

In [ ]:
history = model.fit(
    X_train,y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

In [ ]:
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model loss progress during training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

print("Training Loss: ", training_loss)
print("Validation Loss: ", validation_loss)


# Anomali bulma
*Amaç X_train veri setindeki tahminleri alıp daha sonra bunların gerçek değerler ile ne kadar farklı olduğunu hesaplamaktır Bu farkın mutlak değerinin ortalamasına "mae" yani Mean Absolute Error denir.*
bununla ilgili bir histogram(veri dağılım grafiği çizelim)

In [ ]:
X_train_pred = model.predict(X_train)
train_mae_loss = np.mean(np.abs(X_train_pred - X_train),axis=1)


In [ ]:
import matplotlib.pyplot as plt

plt.hist(train_mae_loss, bins=50)
plt.xlabel("Train MAE loss")
plt.ylabel("Number of Samples");
plt.show()


In [ ]:
plt.plot(train_mae_loss)
plt.xlabel("Sample index")
plt.ylabel("Train MAE loss");
plt.show()


# Eşik Değer seçimi

In [ ]:
THRESHOLD = 0.65

In [ ]:
X_test_pred = model.predict(X_test)
test_mae_loss = np.mean(np.abs(X_test_pred - X_test), axis=1)

In [ ]:
test_score_df = pd.DataFrame(index=test[TIME_STEPS:].index)
test_score_df['loss'] = test_mae_loss
test_score_df['threshold'] = THRESHOLD
test_score_df['anomaly'] = test_score_df.loss > test_score_df.threshold
test_score_df['close'] = test[TIME_STEPS:].close

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(test_score_df.index, test_score_df.loss, label='loss')
plt.plot(test_score_df.index, test_score_df.threshold, label='threshold', color='r')
plt.xticks(rotation=10)
plt.title('Test Loss vs. Threshold')
plt.legend();


In [ ]:
anomalies = test_score_df[test_score_df.anomaly == True]

In [ ]:
plt.figure(figsize=(10, 6))


plt.plot(test_score_df.index, test_score_df['close'], label='close')


anomalies = test_score_df[test_score_df.anomaly == True]


plt.scatter(anomalies.index, anomalies.close, color='r', label='Anomaly')

plt.xticks(rotation=25)
plt.title('Detected Anomalies in the Test Data')
plt.legend()
plt.show()
